# League of Legends Model Testing

This notebook allows you to test your trained LoL model directly without the web interface.

In [7]:
import pandas as pd
import pickle
import os

# Load the trained model
with open('lol_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Load model info
with open('model_info.pkl', 'rb') as f:
    model_info = pickle.load(f)

print(f"Model loaded! Accuracy: {model_info['accuracy']:.3f}")
print(f"Features: {model_info['feature_names']}")

Model loaded! Accuracy: 0.701
Features: ['dragon', 'gold_diff', 'gold_per_kill', 'team_kda', 'enemy_kda']


In [9]:
# Extract model coefficients for JavaScript implementation
print("Model type:", type(model))
print("Pipeline steps:", model.steps if hasattr(model, 'steps') else 'Not a pipeline')

# Get the actual classifier from the pipeline
if hasattr(model, 'steps'):
    classifier = model.steps[-1][1]  # Last step should be the classifier
else:
    classifier = model

print("Classifier type:", type(classifier))
print("Intercept:", classifier.intercept_[0])
print("Coefficients:")
for feature, coef in zip(model_info['feature_names'], classifier.coef_[0]):
    print(f"  {feature}: {coef}")

print("\nJavaScript Implementation Data:")
print("const MODEL_COEFFICIENTS = {")
print(f"  intercept: {classifier.intercept_[0]},")
print("  coefficients: {")
for feature, coef in zip(model_info['feature_names'], classifier.coef_[0]):
    print(f"    {feature}: {coef},")
print("  }")
print("};")

print(f"\nModel accuracy: {model_info['accuracy']:.3f}")
print(f"Feature names: {model_info['feature_names']}")

Model type: <class 'sklearn.pipeline.Pipeline'>
Pipeline steps: [('scaler', StandardScaler()), ('logreg', LogisticRegression(random_state=10, solver='liblinear'))]
Classifier type: <class 'sklearn.linear_model._logistic.LogisticRegression'>
Intercept: 0.16557694182444183
Coefficients:
  dragon: -0.06754431637790291
  gold_diff: 1.107003659509875
  gold_per_kill: -0.04172765965075479
  team_kda: 0.18168310391014147
  enemy_kda: 0.1254660638024227

JavaScript Implementation Data:
const MODEL_COEFFICIENTS = {
  intercept: 0.16557694182444183,
  coefficients: {
    dragon: -0.06754431637790291,
    gold_diff: 1.107003659509875,
    gold_per_kill: -0.04172765965075479,
    team_kda: 0.18168310391014147,
    enemy_kda: 0.1254660638024227,
  }
};

Model accuracy: 0.701
Feature names: ['dragon', 'gold_diff', 'gold_per_kill', 'team_kda', 'enemy_kda']


In [10]:
# Extract scaler parameters
scaler = model.steps[0][1]  # First step is the scaler
print("\nScaler Parameters:")
print("Mean values:", scaler.mean_)
print("Scale values:", scaler.scale_)

print("\nJavaScript Scaler Data:")
print("const SCALER_PARAMS = {")
print("  mean: {")
for feature, mean_val in zip(model_info['feature_names'], scaler.mean_):
    print(f"    {feature}: {mean_val},")
print("  },")
print("  scale: {")
for feature, scale_val in zip(model_info['feature_names'], scaler.scale_):
    print(f"    {feature}: {scale_val},")
print("  }")
print("};")

# Test the model to verify our extraction
test_data = pd.DataFrame({
    'dragon': [1],
    'gold_diff': [7000],
    'gold_per_kill': [3000],
    'team_kda': [2.5],
    'enemy_kda': [1.5]
})

prediction = model.predict_proba(test_data)[0][1]
print(f"\nTest prediction: {prediction:.4f}")


Scaler Parameters:
Mean values: [7.43439227e-01 9.15020718e+01 3.27315758e+03 1.53820032e+00
 1.48669922e+00]
Scale values: [8.31936631e-01 2.00814717e+03 2.30945886e+03 2.44245733e+00
 2.32361947e+00]

JavaScript Scaler Data:
const SCALER_PARAMS = {
  mean: {
    dragon: 0.743439226519337,
    gold_diff: 91.50207182320442,
    gold_per_kill: 3273.1575801210556,
    team_kda: 1.5382003167182776,
    enemy_kda: 1.486699221816108,
  },
  scale: {
    dragon: 0.8319366312774551,
    gold_diff: 2008.1471700613831,
    gold_per_kill: 2309.458855246555,
    team_kda: 2.4424573288528224,
    enemy_kda: 2.3236194700520625,
  }
};

Test prediction: 0.9825


In [11]:
# Verify JavaScript implementation with test case
# Test case: dragon=1, gold_diff=7000, gold_per_kill=3000, team_kda=2.5, enemy_kda=1.5
# This should give us prediction: 0.9825

test_features = {
    'dragon': 1,
    'gold_diff': 7000,
    'gold_per_kill': 3000,
    'team_kda': 2.5,
    'enemy_kda': 1.5
}

print("Verification Test Case:")
print("Features:", test_features)

# Calculate what JavaScript will do step by step
print("\nJavaScript Implementation Steps:")

# 1. Scale features
scaled_features = {}
for feature, value in test_features.items():
    mean_val = scaler.mean_[model_info['feature_names'].index(feature)]
    scale_val = scaler.scale_[model_info['feature_names'].index(feature)]
    scaled_val = (value - mean_val) / scale_val
    scaled_features[feature] = scaled_val
    print(f"{feature}: ({value} - {mean_val}) / {scale_val} = {scaled_val}")

# 2. Calculate logits
logits = classifier.intercept_[0]
print(f"\nLogits calculation:")
print(f"Start with intercept: {logits}")

for feature, scaled_val in scaled_features.items():
    coef = classifier.coef_[0][model_info['feature_names'].index(feature)]
    contribution = scaled_val * coef
    logits += contribution
    print(f"+ {feature}: {scaled_val} * {coef} = {contribution}")

print(f"Final logits: {logits}")

# 3. Apply sigmoid
import math
sigmoid = 1 / (1 + math.exp(-logits))
print(f"Sigmoid(logits): 1 / (1 + exp(-{logits})) = {sigmoid}")

# Compare with actual model
test_data = pd.DataFrame([test_features])
actual_prediction = model.predict_proba(test_data)[0][1]
print(f"\nActual model prediction: {actual_prediction}")
print(f"JavaScript prediction: {sigmoid}")
print(f"Difference: {abs(actual_prediction - sigmoid)}")

if abs(actual_prediction - sigmoid) < 0.0001:
    print("✅ JavaScript implementation matches Python model!")
else:
    print("❌ JavaScript implementation differs from Python model")

Verification Test Case:
Features: {'dragon': 1, 'gold_diff': 7000, 'gold_per_kill': 3000, 'team_kda': 2.5, 'enemy_kda': 1.5}

JavaScript Implementation Steps:
dragon: (1 - 0.743439226519337) / 0.8319366312774551 = 0.3083898025823299
gold_diff: (7000 - 91.50207182320442) / 2008.1471700613831 = 3.440234874800348
gold_per_kill: (3000 - 3273.1575801210556) / 2309.458855246555 = -0.11827774264109749
team_kda: (2.5 - 1.5382003167182776) / 2.4424573288528224 = 0.39378361780161053
enemy_kda: (1.5 - 1.486699221816108) / 2.3236194700520625 = 0.005724163683132707

Logits calculation:
Start with intercept: 0.16557694182444183
+ dragon: 0.3083898025823299 * -0.06754431637790291 = -0.02082997839333991
+ gold_diff: 3.440234874800348 * 1.107003659509875 = 3.8083525959774813
+ gold_per_kill: -0.11827774264109749 * -0.04172765965075479 = 0.004935453389187283
+ team_kda: 0.39378361780161053 * 0.18168310391014147 = 0.07154382995116144
+ enemy_kda: 0.005724163683132707 * 0.1254660638024227 = 0.000718188285

In [14]:
# Test the new model.py by importing it directly
import sys
sys.path.append('.')

try:
    from model import LoLWinPredictor
    
    print("✅ Successfully imported LoLWinPredictor")
    
    # Create predictor instance
    predictor = LoLWinPredictor()
    
    if predictor.is_loaded:
        print("✅ Model loaded successfully")
        
        # Test prediction
        result = predictor.predict(
            team_kills=15, team_deaths=8, team_assists=22,
            enemy_kills=8, enemy_deaths=15, enemy_assists=12,
            team_gold=45000, enemy_gold=38000, dragon_acquisition='team'
        )
        
        if result['success']:
            print("✅ Prediction successful!")
            print(f"Win Probability: {result['win_probability']}%")
            print(f"Prediction: {result['prediction_label']}")
            print(f"Model Accuracy: {result['model_info']['accuracy']}")
            
            # Compare with original notebook function
            print("\nComparing with notebook function:")
            test_prediction(15, 8, 22, 8, 15, 12, 45000, 38000, 'team')
        else:
            print(f"❌ Prediction failed: {result['error']}")
    else:
        print("❌ Model failed to load")
        
except ImportError as e:
    print(f"❌ Import error: {e}")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Successfully imported LoLWinPredictor
✅ Model loaded successfully from lol_model.pkl
📊 Model accuracy: 0.701
🔧 Features: ['dragon', 'gold_diff', 'gold_per_kill', 'team_kda', 'enemy_kda']
✅ Model loaded successfully
✅ Prediction successful!
Win Probability: 98.48%
Prediction: WIN
Model Accuracy: 0.701

Comparing with notebook function:
INPUT STATISTICS:
Your Team:   15K/8D/22A | 8 gold
Enemy Team:  15K/12D/45000A | 38,000 gold
Dragons:     Team

CALCULATED FEATURES:
Team KDA:        4.62
Enemy KDA:       3751.25
Gold Difference: -37,992
Gold per Kill:   1267
Dragon Value:    1

PREDICTION:
Win Probability: 100.0%
Result:          WIN


In [5]:
def test_prediction(team_kills, team_deaths, team_assists, team_gold,
                   enemy_kills, enemy_deaths, enemy_assists, enemy_gold,
                   dragon_acquisition='none'):
    """
    Test the model with custom inputs
    dragon_acquisition: 'team', 'enemy', or 'none'
    """
    
    # Calculate features exactly like the model expects
    team_kda = (team_kills + team_assists) / max(1, team_deaths)
    enemy_kda = (enemy_kills + enemy_assists) / max(1, enemy_deaths)
    gold_diff = team_gold - enemy_gold
    total_kills = team_kills + enemy_kills
    gold_per_kill = (team_gold + enemy_gold) / max(1, total_kills)
    
    # Convert dragon to numeric
    dragon = 1 if dragon_acquisition.lower() == 'team' else 0
    
    # Create input dataframe
    input_data = pd.DataFrame({
        'dragon': [dragon],
        'gold_diff': [gold_diff],
        'gold_per_kill': [gold_per_kill],
        'team_kda': [team_kda],
        'enemy_kda': [enemy_kda]
    })
    
    # Get prediction
    win_probability = model.predict_proba(input_data)[0][1]
    prediction = model.predict(input_data)[0]
    
    # Display results
    print("=" * 50)
    print("INPUT STATISTICS:")
    print(f"Your Team:   {team_kills}K/{team_deaths}D/{team_assists}A | {team_gold:,} gold")
    print(f"Enemy Team:  {enemy_kills}K/{enemy_deaths}D/{enemy_assists}A | {enemy_gold:,} gold")
    print(f"Dragons:     {dragon_acquisition.title()}")
    
    print("\nCALCULATED FEATURES:")
    print(f"Team KDA:        {team_kda:.2f}")
    print(f"Enemy KDA:       {enemy_kda:.2f}")
    print(f"Gold Difference: {gold_diff:+,}")
    print(f"Gold per Kill:   {gold_per_kill:.0f}")
    print(f"Dragon Value:    {dragon}")
    
    print("\nPREDICTION:")
    print(f"Win Probability: {win_probability*100:.1f}%")
    print(f"Result:          {'WIN' if prediction == 1 else 'LOSE'}")
    print("=" * 50)
    
    return win_probability, prediction

## Test Scenarios

Run different game scenarios to see how the model responds:

In [12]:
# Test 1: Winning scenario
print("TEST 1: Winning Game")
test_prediction(
    team_kills=15, team_deaths=8, team_assists=22, team_gold=45000,
    enemy_kills=8, enemy_deaths=15, enemy_assists=12, enemy_gold=38000,
    dragon_acquisition='team'
)

TEST 1: Winning Game
INPUT STATISTICS:
Your Team:   15K/8D/22A | 45,000 gold
Enemy Team:  8K/15D/12A | 38,000 gold
Dragons:     Team

CALCULATED FEATURES:
Team KDA:        4.62
Enemy KDA:       1.33
Gold Difference: +7,000
Gold per Kill:   3609
Dragon Value:    1

PREDICTION:
Win Probability: 98.5%
Result:          WIN


(np.float64(0.9847584122845884), np.int64(1))

In [ ]:
# Test 2: Losing scenario
print("TEST 2: Losing Game")
test_prediction(
    team_kills=8, team_deaths=15, team_assists=12, team_gold=38000,
    enemy_kills=15, enemy_deaths=8, enemy_assists=22, enemy_gold=45000,
    dragon_acquisition='enemy'
)

In [ ]:
# Test 3: Balanced game
print("TEST 3: Balanced Game")
test_prediction(
    team_kills=12, team_deaths=12, team_assists=18, team_gold=42000,
    enemy_kills=12, enemy_deaths=12, enemy_assists=18, enemy_gold=42000,
    dragon_acquisition='none'
)

In [ ]:
# Test 4: Custom scenario - modify these values!
print("TEST 4: Custom Scenario")
test_prediction(
    team_kills=20,     # Your team kills
    team_deaths=10,    # Your team deaths
    team_assists=25,   # Your team assists
    team_gold=50000,   # Your team gold
    enemy_kills=15,    # Enemy kills
    enemy_deaths=20,   # Enemy deaths
    enemy_assists=20,  # Enemy assists
    enemy_gold=48000,  # Enemy gold
    dragon_acquisition='team'  # 'team', 'enemy', or 'none'
)

## Feature Importance Analysis

See which features have the most impact on predictions:

In [ ]:
# Display feature importance
import matplotlib.pyplot as plt

feature_importance = pd.Series(model_info['feature_importance'])
feature_importance_sorted = feature_importance.abs().sort_values(ascending=True)

plt.figure(figsize=(10, 6))
feature_importance_sorted.plot(kind='barh')
plt.title('Feature Importance (Absolute Values)')
plt.xlabel('Coefficient Magnitude')
plt.tight_layout()
plt.show()

print("Feature Coefficients:")
for feature, coef in feature_importance.sort_values(key=abs, ascending=False).items():
    direction = "helps win" if coef > 0 else "helps lose"
    print(f"{feature:15s}: {coef:+.3f} ({direction})")

## Comprehensive Model Analysis

Let's create multiple visualizations to understand how the model behaves across different scenarios:

In [ ]:
# 1. Feature Importance Visualization (Enhanced)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style for better looking plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Horizontal bar chart
feature_importance = pd.Series(model_info['feature_importance'])
colors = ['#d62728' if x < 0 else '#2ca02c' for x in feature_importance.values]  # Using distinct colors
feature_importance.plot(kind='barh', ax=ax1, color=colors)
ax1.set_title('Feature Coefficients (Positive = Win, Negative = Loss)')
ax1.set_xlabel('Coefficient Value')
ax1.axvline(x=0, color='black', linestyle='--', alpha=0.7)

# Absolute importance
feature_importance.abs().sort_values().plot(kind='barh', ax=ax2, color='#1f77b4')
ax2.set_title('Feature Importance (Absolute Values)')
ax2.set_xlabel('Absolute Coefficient Magnitude')

plt.tight_layout()
plt.show()

print("Feature Analysis:")
for feature, coef in feature_importance.sort_values(key=abs, ascending=False).items():
    direction = "[+] Helps Win" if coef > 0 else "[-] Helps Lose"
    print(f"{feature:15s}: {coef:+.4f} ({direction})")

In [ ]:
# 2. Win Probability Heatmaps
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Generate test data ranges
team_kdas = np.linspace(0.5, 5.0, 20)
enemy_kdas = np.linspace(0.5, 5.0, 20)
gold_diffs = np.linspace(-10000, 10000, 20)

# Heatmap 1: Team KDA vs Enemy KDA
kda_probs = np.zeros((len(team_kdas), len(enemy_kdas)))
for i, team_kda in enumerate(team_kdas):
    for j, enemy_kda in enumerate(enemy_kdas):
        input_data = pd.DataFrame({
            'dragon': [0],
            'gold_diff': [0],
            'gold_per_kill': [3000],
            'team_kda': [team_kda],
            'enemy_kda': [enemy_kda]
        })
        kda_probs[i, j] = model.predict_proba(input_data)[0][1]

im1 = axes[0,0].imshow(kda_probs, cmap='viridis', aspect='auto')
axes[0,0].set_title('Win Probability: Team KDA vs Enemy KDA')
axes[0,0].set_xlabel('Enemy KDA')
axes[0,0].set_ylabel('Team KDA')
axes[0,0].set_xticks(range(0, 20, 4))
axes[0,0].set_xticklabels([f'{enemy_kdas[i]:.1f}' for i in range(0, 20, 4)])
axes[0,0].set_yticks(range(0, 20, 4))
axes[0,0].set_yticklabels([f'{team_kdas[i]:.1f}' for i in range(0, 20, 4)])
plt.colorbar(im1, ax=axes[0,0])

# Heatmap 2: Gold Difference vs Team KDA
gold_kda_probs = np.zeros((len(gold_diffs), len(team_kdas)))
for i, gold_diff in enumerate(gold_diffs):
    for j, team_kda in enumerate(team_kdas):
        input_data = pd.DataFrame({
            'dragon': [0],
            'gold_diff': [gold_diff],
            'gold_per_kill': [3000],
            'team_kda': [team_kda],
            'enemy_kda': [2.0]
        })
        gold_kda_probs[i, j] = model.predict_proba(input_data)[0][1]

im2 = axes[0,1].imshow(gold_kda_probs, cmap='viridis', aspect='auto')
axes[0,1].set_title('Win Probability: Gold Difference vs Team KDA')
axes[0,1].set_xlabel('Team KDA')
axes[0,1].set_ylabel('Gold Difference')
axes[0,1].set_xticks(range(0, 20, 4))
axes[0,1].set_xticklabels([f'{team_kdas[i]:.1f}' for i in range(0, 20, 4)])
axes[0,1].set_yticks(range(0, 20, 4))
axes[0,1].set_yticklabels([f'{gold_diffs[i]:.0f}' for i in range(0, 20, 4)])
plt.colorbar(im2, ax=axes[0,1])

# Line Plot 3: Win Probability vs Gold Difference
gold_range = np.linspace(-15000, 15000, 100)
gold_probs = []
for gold_diff in gold_range:
    input_data = pd.DataFrame({
        'dragon': [0],
        'gold_diff': [gold_diff],
        'gold_per_kill': [3000],
        'team_kda': [2.0],
        'enemy_kda': [2.0]
    })
    gold_probs.append(model.predict_proba(input_data)[0][1])

axes[1,0].plot(gold_range, gold_probs, linewidth=3, color='#1f77b4')
axes[1,0].set_title('Win Probability vs Gold Difference')
axes[1,0].set_xlabel('Gold Difference')
axes[1,0].set_ylabel('Win Probability')
axes[1,0].axhline(y=0.5, color='#ff7f0e', linestyle='--', alpha=0.7, label='50% Win Rate')
axes[1,0].axvline(x=0, color='gray', linestyle='--', alpha=0.7, label='Even Gold')
axes[1,0].grid(True, alpha=0.3)
axes[1,0].legend()

# Line Plot 4: Win Probability vs KDA Ratio
kda_ratio_range = np.linspace(0.2, 8.0, 100)
kda_ratio_probs = []
for kda_ratio in kda_ratio_range:
    input_data = pd.DataFrame({
        'dragon': [0],
        'gold_diff': [0],
        'gold_per_kill': [3000],
        'team_kda': [kda_ratio],
        'enemy_kda': [2.0]
    })
    kda_ratio_probs.append(model.predict_proba(input_data)[0][1])

axes[1,1].plot(kda_ratio_range, kda_ratio_probs, linewidth=3, color='#2ca02c')
axes[1,1].set_title('Win Probability vs Team KDA')
axes[1,1].set_xlabel('Team KDA')
axes[1,1].set_ylabel('Win Probability')
axes[1,1].axhline(y=0.5, color='#ff7f0e', linestyle='--', alpha=0.7, label='50% Win Rate')
axes[1,1].axvline(x=2.0, color='gray', linestyle='--', alpha=0.7, label='Enemy KDA (2.0)')
axes[1,1].grid(True, alpha=0.3)
axes[1,1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# 3. Dragon Impact Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Test different scenarios with and without dragon
scenarios = [
    {'name': 'Losing KDA', 'team_kda': 1.5, 'enemy_kda': 3.0, 'gold_diff': -3000},
    {'name': 'Even Game', 'team_kda': 2.0, 'enemy_kda': 2.0, 'gold_diff': 0},
    {'name': 'Winning KDA', 'team_kda': 3.0, 'enemy_kda': 1.5, 'gold_diff': 3000},
    {'name': 'Dominating', 'team_kda': 4.0, 'enemy_kda': 1.0, 'gold_diff': 5000}
]

no_dragon_probs = []
with_dragon_probs = []
scenario_names = []

for scenario in scenarios:
    scenario_names.append(scenario['name'])
    
    # Without dragon
    input_data = pd.DataFrame({
        'dragon': [0],
        'gold_diff': [scenario['gold_diff']],
        'gold_per_kill': [3000],
        'team_kda': [scenario['team_kda']],
        'enemy_kda': [scenario['enemy_kda']]
    })
    no_dragon_probs.append(model.predict_proba(input_data)[0][1])
    
    # With dragon
    input_data = pd.DataFrame({
        'dragon': [1],
        'gold_diff': [scenario['gold_diff']],
        'gold_per_kill': [3000],
        'team_kda': [scenario['team_kda']],
        'enemy_kda': [scenario['enemy_kda']]
    })
    with_dragon_probs.append(model.predict_proba(input_data)[0][1])

x = np.arange(len(scenario_names))
width = 0.35

bars1 = ax1.bar(x - width/2, no_dragon_probs, width, label='No Dragon', alpha=0.8, color='#ff7f0e')
bars2 = ax1.bar(x + width/2, with_dragon_probs, width, label='With Dragon', alpha=0.8, color='#2ca02c')

ax1.set_xlabel('Game Scenarios')
ax1.set_ylabel('Win Probability')
ax1.set_title('Dragon Impact Across Different Game States')
ax1.set_xticks(x)
ax1.set_xticklabels(scenario_names, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.2f}', ha='center', va='bottom', fontsize=9)

for bar in bars2:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.2f}', ha='center', va='bottom', fontsize=9)

# Dragon advantage calculation
dragon_advantages = [with_dragon_probs[i] - no_dragon_probs[i] for i in range(len(scenarios))]
bars3 = ax2.bar(scenario_names, dragon_advantages, color='#ffbb78', alpha=0.8)
ax2.set_xlabel('Game Scenarios')
ax2.set_ylabel('Dragon Advantage (Probability Increase)')
ax2.set_title('Dragon Advantage by Game State')
ax2.set_xticklabels(scenario_names, rotation=45)
ax2.grid(True, alpha=0.3)

# Add value labels
for bar in bars3:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.001,
             f'{height:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("Dragon Impact Analysis:")
for i, scenario in enumerate(scenarios):
    advantage = dragon_advantages[i]
    print(f"{scenario['name']:12s}: Dragon adds {advantage:+.3f} ({advantage*100:+.1f}%) win probability")

In [ ]:
# 4. Gold Efficiency Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gold per kill impact
gold_per_kill_range = np.linspace(1500, 5000, 50)
gold_efficiency_probs = []

for gpk in gold_per_kill_range:
    input_data = pd.DataFrame({
        'dragon': [0],
        'gold_diff': [0],
        'gold_per_kill': [gpk],
        'team_kda': [2.0],
        'enemy_kda': [2.0]
    })
    gold_efficiency_probs.append(model.predict_proba(input_data)[0][1])

ax1.plot(gold_per_kill_range, gold_efficiency_probs, linewidth=3, color='#9467bd')
ax1.set_title('Win Probability vs Gold Efficiency')
ax1.set_xlabel('Gold per Kill')
ax1.set_ylabel('Win Probability')
ax1.axhline(y=0.5, color='#ff7f0e', linestyle='--', alpha=0.7, label='50% Win Rate')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Feature interaction: Gold difference vs different KDA scenarios
gold_range = np.linspace(-10000, 10000, 50)
kda_scenarios = [
    {'label': 'Low KDA (1.0 vs 3.0)', 'team_kda': 1.0, 'enemy_kda': 3.0, 'color': '#d62728'},
    {'label': 'Even KDA (2.0 vs 2.0)', 'team_kda': 2.0, 'enemy_kda': 2.0, 'color': '#1f77b4'},
    {'label': 'High KDA (3.0 vs 1.0)', 'team_kda': 3.0, 'enemy_kda': 1.0, 'color': '#2ca02c'}
]

for scenario in kda_scenarios:
    probs = []
    for gold_diff in gold_range:
        input_data = pd.DataFrame({
            'dragon': [0],
            'gold_diff': [gold_diff],
            'gold_per_kill': [3000],
            'team_kda': [scenario['team_kda']],
            'enemy_kda': [scenario['enemy_kda']]
        })
        probs.append(model.predict_proba(input_data)[0][1])
    
    ax2.plot(gold_range, probs, linewidth=3, color=scenario['color'], label=scenario['label'])

ax2.set_title('Gold Impact Across Different KDA Scenarios')
ax2.set_xlabel('Gold Difference')
ax2.set_ylabel('Win Probability')
ax2.axhline(y=0.5, color='#ff7f0e', linestyle='--', alpha=0.7)
ax2.axvline(x=0, color='gray', linestyle='--', alpha=0.7)
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 6. Sensitivity Analysis - How much do small changes affect predictions?
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Base scenario
base_scenario = {
    'team_kda': 2.5,
    'enemy_kda': 2.0,
    'gold_diff': 2000,
    'gold_per_kill': 3000,
    'dragon': 1
}

# Get base prediction
input_data = pd.DataFrame({
    'dragon': [base_scenario['dragon']],
    'gold_diff': [base_scenario['gold_diff']],
    'gold_per_kill': [base_scenario['gold_per_kill']],
    'team_kda': [base_scenario['team_kda']],
    'enemy_kda': [base_scenario['enemy_kda']]
})
base_prob = model.predict_proba(input_data)[0][1]

# Test sensitivity to each feature
features_to_test = ['team_kda', 'enemy_kda', 'gold_diff', 'gold_per_kill']
sensitivity_ranges = {
    'team_kda': np.linspace(1.0, 4.0, 50),
    'enemy_kda': np.linspace(1.0, 4.0, 50),
    'gold_diff': np.linspace(-5000, 8000, 50),
    'gold_per_kill': np.linspace(2000, 4000, 50)
}

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for i, feature in enumerate(features_to_test):
    probabilities = []
    
    for value in sensitivity_ranges[feature]:
        scenario_copy = base_scenario.copy()
        scenario_copy[feature] = value
        
        input_data = pd.DataFrame({
            'dragon': [scenario_copy['dragon']],
            'gold_diff': [scenario_copy['gold_diff']],
            'gold_per_kill': [scenario_copy['gold_per_kill']],
            'team_kda': [scenario_copy['team_kda']],
            'enemy_kda': [scenario_copy['enemy_kda']]
        })
        probabilities.append(model.predict_proba(input_data)[0][1])
    
    ax1.plot(sensitivity_ranges[feature], probabilities, 
             label=feature.replace('_', ' ').title(), 
             linewidth=2, color=colors[i])

ax1.axhline(y=base_prob, color='black', linestyle='--', alpha=0.7, label=f'Base Scenario ({base_prob:.3f})')
ax1.set_title('Feature Sensitivity Analysis')
ax1.set_xlabel('Feature Value')
ax1.set_ylabel('Win Probability')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Calculate feature sensitivity (max change from base)
sensitivities = []
feature_labels = []

for feature in features_to_test:
    probabilities = []
    
    for value in sensitivity_ranges[feature]:
        scenario_copy = base_scenario.copy()
        scenario_copy[feature] = value
        
        input_data = pd.DataFrame({
            'dragon': [scenario_copy['dragon']],
            'gold_diff': [scenario_copy['gold_diff']],
            'gold_per_kill': [scenario_copy['gold_per_kill']],
            'team_kda': [scenario_copy['team_kda']],
            'enemy_kda': [scenario_copy['enemy_kda']]
        })
        probabilities.append(model.predict_proba(input_data)[0][1])
    
    max_change = max(abs(np.array(probabilities) - base_prob))
    sensitivities.append(max_change)
    feature_labels.append(feature.replace('_', ' ').title())

bars = ax2.bar(feature_labels, sensitivities, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'], alpha=0.7)
ax2.set_title('Feature Sensitivity Ranking')
ax2.set_ylabel('Maximum Probability Change')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Add value labels
for bar, sensitivity in zip(bars, sensitivities):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.005,
             f'{sensitivity:.3f}', ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()

print(f"\nSensitivity Analysis Results (Base scenario: {base_prob:.3f} win probability):")
for feature, sensitivity in zip(feature_labels, sensitivities):
    print(f"{feature:15s}: Max change of ±{sensitivity:.3f} ({sensitivity*100:.1f}%)")